# Julia code for ACE basis evaluation

## Misc

In [2]:
include("misc.jl")

parse_dict (generic function with 1 method)

In [3]:
# full path not necessary if .json file is in the same dir
string_dict = JSON.parsefile("evACE.json")
symbol_dict = parse_dict("evACE.json")

Dict{Symbol, Dict{Symbol}} with 3 entries:
  :evaluation => Dict(:c_atom=>"Li")
  :trajectory => Dict(:index=>"0:100:1", :traj_file=>"./examples/traj_2.1_0-100…
  :ace_basis  => Dict{Symbol, Any}(:species=>Any["H", "C", "O", "F", "P", "Li",…

In [4]:
symbol_dict[:trajectory]

Dict{Symbol, String} with 2 entries:
  :index     => "0:100:1"
  :traj_file => "./examples/traj_2.1_0-100-1_newatoms.xyz"

## Gen ACE basis

In [24]:
include("descriptor.jl")

ACEdescriptor (generic function with 1 method)

In [7]:
ace_basis = dict_to_ace_basis(symbol_dict)

RPIBasis{Float64, BasicPSH1pBasis{Float64, 8, ACE1.OrthPolys.TransformedPolys{Float64, PolyTransform{Int64, Float64}, ACE1.OrthPolys.OrthPolyBasis{Float64}}}, 8, ACE1.DAGEvaluator}(PIBasis{BasicPSH1pBasis{Float64, 8, ACE1.OrthPolys.TransformedPolys{Float64, PolyTransform{Int64, Float64}, ACE1.OrthPolys.OrthPolyBasis{Float64}}}, 8, ACE1.DAGEvaluator}(BasicPSH1pBasis{Float64, 8, ACE1.OrthPolys.TransformedPolys{Float64, PolyTransform{Int64, Float64}, ACE1.OrthPolys.OrthPolyBasis{Float64}}}(ACE1.OrthPolys.TransformedPolys{Float64, PolyTransform{Int64, Float64}, ACE1.OrthPolys.OrthPolyBasis{Float64}}(ACE1.OrthPolys.OrthPolyBasis{Float64}(2, 0.053254437869822494, 2, 1.8595041322314048, [2.358083222149332, 3.6723879325307336, 2.9607738063804043, 2.6916125512549365], [0.0, -3.512195745342244, -2.831622736142675, -2.5742024874024536], [0.0, 0.0, -0.8062257748298562, -0.9090909090909164], [0.05415756271700328, 0.05596381241136487, 0.057770062105726445, 0.05957631180008803, 0.061382561494449615, 

In [8]:
length(ace_basis)

5120

In [9]:
# input to generate the ACE basis
# -> some parameters are place by default and other are hidden
symbol_dict[:ace_basis]

Dict{Symbol, Any} with 7 entries:
  :species => ["H", "C", "O", "F", "P", "Li", "V", "Sc"]
  :N       => 3
  :maxdeg  => 4
  :rin     => 0.1
  :rcut    => 5.5
  :pin     => 2
  :r0      => 0.5

## Load the trajectory

In [10]:
include("trajectory.jl")

_traj_init (generic function with 1 method)

In [11]:
symbol_dict[:trajectory]

Dict{Symbol, String} with 2 entries:
  :index     => "0:100:1"
  :traj_file => "./examples/traj_2.1_0-100-1_newatoms.xyz"

In [12]:
# read the trajectory
traj_db = read_xyz(; symbol_dict[:trajectory]...)

┌ Info: Keys used: E => "", F => "", V => ""
└ @ IPFitting.Data /home/agardin/.julia/packages/IPFitting/E09UD/src/data.jl:152
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


┌─────────────┬───────┬────────┬───────┬─────────┬───────┐
│ config_type │ #cfgs │  #envs │    #E │      #F │    #V │
│      String │ Int64 │  Int64 │ Int64 │   Int64 │ Int64 │
├─────────────┼───────┼────────┼───────┼─────────┼───────┤
│     nothing │   100 │ 840200 │     0 │       0 │     0 │
├─────────────┼───────┼────────┼───────┼─────────┼───────┤
│       total │   100 │ 840200 │     0 │       0 │     0 │
│     missing │     0 │      0 │   100 │ 2520600 │   900 │
└─────────────┴───────┴────────┴───────┴─────────┴───────┘


100-element Vector{Dat}:
 Dat(Atoms{Float64}(StaticArraysCore.SVector{3, Float64}[[13.4214334, 37.9411774, 25.6024189], [12.4229565, 39.045311, 25.9095592], [14.2894106, 38.552597, 24.6148567], [12.5934677, 39.9337463, 24.7763767], [13.7624607, 39.6994514, 24.1646919], [12.8664417, 37.092144, 25.1768265], [14.003438, 37.619648, 26.4787102], [12.6682644, 39.5936661, 26.8310394], [11.3863182, 38.6841393, 25.8393097], [14.3525457, 40.5229683, 23.5030918]  …  [28.7237568, 37.9477577, 18.0548611], [16.9131069, 28.2505417, 28.0804462], [44.8431015, 22.4488029, 19.3294239], [40.0488434, 42.6378517, 27.3642979], [18.8909836, 43.4313812, 1.53649151], [30.3799152, 30.5201054, 44.2985649], [21.5288963, 0.73614603, 40.0230713], [8.79868984, 37.8729858, 21.1588421], [44.5085945, 33.5317574, 24.1566582], [35.9943466, 7.74214506, 11.2953444]], StaticArraysCore.SVector{3, Float64}[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.

In [13]:
length(traj_db)

100

### ACE evaluation

In [15]:
central_Z = findall(x->x==symbol_dict[:evaluation][:c_atom], 
                    elements_dict["symbols"]
                    )

1-element Vector{Int64}:
 3

In [20]:
eval_sites = findall(x->x==central_Z[1], traj_db[1].at.Z)
println(length(eval_sites))

114


In [29]:
single_site = ACE1pack.JuLIP.site_energy(ace_basis, traj_db[1].at, eval_sites[1])
println(length(single_site))

5120


In [33]:
single_conf = [ACE1pack.JuLIP.site_energy(ace_basis, traj_db[1].at, site) for site in eval_sites]
println(length(single_conf), " ", length(single_conf[1]))

114 5120


In [34]:
typeof(single_conf)

Vector{Vector{Float64}} (alias for Array{Array{Float64, 1}, 1})

In [36]:
ace_descr = [[ACE1pack.JuLIP.site_energy(ace_basis, at_conf.at, site) for site in eval_sites] for at_conf in traj_db]

100-element Vector{Vector{Vector{Float64}}}:
 [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.

In [37]:
println(length(ace_descr))

100


In [26]:
atZ = [convert(Int, Z) for Z in unique(sort(traj_init[1].at.Z))]

8-element Vector{Int64}:
  1
  3
  6
  8
  9
 15
 21
 23

In [32]:
central_Z = findall(x->x==symbol_dict[:evaluation][:c_atom], 
                    elements_dict["symbols"])

1-element Vector{Int64}:
 3

In [14]:
symbol_dict[:evaluation]

Dict{Symbol, String} with 1 entry:
  :c_atom => "Li"